In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [2]:
graph, random_solution, total_edge_length = dg.read_instance('data\data8.txt')
num_nodes = len(graph)

In [3]:
def tabu_swap_neighbors(solution):
    #for each pair of solution elements that are next to eachother, create a solution where they are swaapped
    neighbors = []

    for i in range(len(solution) - 1):
        neighbor = solution.copy()
        #this works in python
        neighbor[i], neighbor[i + 1] = neighbor[i + 1], neighbor[i]
        neighbors.append(neighbor)
        
    return neighbors

In [4]:
def tabu_swap(solution):
    #this just creates all possible swap combos, every element is swapped with every other element
    #still polynomial, and not even that much slower, even for large N
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            solution = solution.copy()
            tmp = solution[i]
            solution[i] = solution[j]
            solution[j] = tmp
            solutions.append(solution)
            
    return solutions

In [5]:
def tabu_inverse_complete(solution):
    #creates all possible solutions based on inversing every possible part of the starting solution
    #seems overkill but - surprisingly good time and results
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            new_solution = solution.copy()
            new_solution[i:j] = reversed(new_solution[i:j])
            solutions.append(new_solution)
            
    return solutions

In [6]:
def tabu_inverse(solution):
    #simpler version of the previous method:
    #for every position i in the solution, get a random index j after it
    #then reverse the solution from i to j
    solutions = []

    for i in range(len(solution) - 1):
        new_solution = solution.copy()
        j = random.randrange(i+1, len(solution))
        new_solution[i:j] = reversed(new_solution[i:j])
        solutions.append(new_solution)
        
    return solutions

In [7]:
def tabu_scramble(solution):
    #every time we scramble a part of the solution - we create a new solution
    #but keep the original unchanged - the neighbourhood stems exclusivly from the original solution
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            new_solution = solution.copy()
            sub = new_solution[i:j]
            random.shuffle(sub)
            new_solution[i:j] = sub
            solutions.append(new_solution)
            
    return solutions

In [8]:
def tabu_scramble_continuous(solution):
    #a bit more experimental, here we keep continously changing the solution - it fits the term "neighbourhood" less
    #but it widens the search
    solutions = []

    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            solution = solution.copy()
            sub = solution[i:j]
            random.shuffle(sub)
            solution[i:j] = sub
            solutions.append(solution)
            
    return solutions

In [10]:
def tabu_search(graph, solution, max_iterations, neighbor_gen_func, tabu_tenure):
    num_nodes = len(graph)
    current_solution = deepcopy(solution)
    current_value = dg.calculate_total_edge_length(graph, current_solution)
    best_solution = current_solution.copy()
    best_value = current_value
    tabu_list = []

    best_i = -1

    for ind in range(max_iterations):
        neighbors = neighbor_gen_func(current_solution)
        neighbors.sort(key=lambda solution: dg.calculate_total_edge_length(graph, solution))
        #print(neighbors[0], calculate_total_edge_length(graph, neighbors[0]))

        next_solution = None
        for neighbor in neighbors:
            if neighbor not in tabu_list:
                next_solution = neighbor
                break

        #if all neighbors are tabu, choose the best one
        if next_solution is None:
            next_solution = neighbors[0]

        #update tabu list
        #tenure decides how long already visited solutions will stay forbidden
        tabu_list.append(next_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        #update current solution
        current_solution = deepcopy(next_solution)
        current_value = dg.calculate_total_edge_length(graph, current_solution)

        #update best solution
        if current_value < best_value:
            best_i = ind
            best_solution = deepcopy(current_solution)
            best_value = current_value

    return best_solution, best_value, best_i

In [11]:
sols = []
values = []
iters = []
times = []

In [12]:
start = time.time()
solution, value, iter = tabu_search(graph, random_solution, 500, tabu_swap_neighbors, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 2, 0, 7, 4, 6, 3, 1] 21 15


In [13]:
start = time.time()
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_swap, 10)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 4, 7, 0, 2, 5, 6] 21 5


In [14]:
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_inverse_complete, 20)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 4, 7, 0, 2, 5, 6] 21 4


In [15]:
solution, value, iter = tabu_search(graph, random_solution, 500, tabu_inverse, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 7, 4, 0, 5, 2, 6] 21 38


In [16]:
solution, value, iter = tabu_search(graph, random_solution, 200, tabu_scramble, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 4, 7, 0, 5, 2, 6] 21 34


In [17]:
solution, value, iter = tabu_search(graph, random_solution, 300, tabu_scramble_continuous, 5)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 3, 7, 4, 0, 5, 2, 6] 21 49


In [19]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [tabu_swap_neighbors, tabu_swap, tabu_inverse_complete, tabu_inverse, tabu_scramble, tabu_scramble_continuous]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1))  
df.to_csv('comparison_tables/tabu_search.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/tabu_search.csv'), index=False)

,Dim,Method,Value,Time
0,8,tabu_swap_neighbors,21,0.06
1,8,tabu_swap,21,0.07
2,8,tabu_inverse_complete,21,0.15
3,8,tabu_inverse,21,0.24
4,8,tabu_scramble,21,0.35
5,8,tabu_scramble_continuous,21,0.57
6,8,tabu_double_bridge_move,23,0.61


In [20]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: tabu_swap_neighbors 21 0.06 15


,Dim,Method,Value,Time,Best_Iter
0,8,tabu_swap_neighbors,21,0.06,15


In [21]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    
    df.loc[row_to_update.index, 'tabu'] = values[best_i]
    df.loc[row_to_update.index, 'tabu_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'tabu': values[best_i], 'tabu_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [22]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [21.29, 0.29, 43.71]


,Dim,Value,Time,Best_Iter
0,8,21.29,0.29,43.71


In [23]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:

    df.loc[row_to_update.index, 'tabu'] = average[0]
    df.loc[row_to_update.index, 'tabu_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'tabu': values[best_i], 'tabu_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/averages.csv', index=False)